In [ ]:
!pip install insightface onnxruntime-gpu

In [ ]:
import cv2
import shutil
import numpy as np
import os
import zipfile
import tempfile
import random
import gc
from pathlib import Path
from tqdm import tqdm
from insightface.app import FaceAnalysis
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [ ]:
ZIP_PATH = "/content/drive/MyDrive/Celeb-DF-v2.zip"
SAVE_BASE_PATH = "/content/drive/MyDrive/HECTO/Dataset/Celeb_frames_ver2"
NUM_FRAMES = 15
TARGET_SIZE = (256, 256)
FAKE_SAMPLE_COUNT = 890

In [ ]:
# providers=['CUDAExecutionProvider']로 GPU 강제 사용
detector = FaceAnalysis(allowed_modules=['detection', 'landmark_2d'], providers=['CUDAExecutionProvider'])
detector.prepare(ctx_id=0, det_size=(640, 640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/genderage.onnx genderage
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
model ignore: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition
set det-size: (640, 640)


In [ ]:
def get_hybrid_face(image, face_info, target_size=(256, 256)):
    try:
        h, w = image.shape[:2]
        bbox = face_info.bbox.astype(int)
        landmarks = getattr(face_info, 'kps', None)

        cx = (bbox[0] + bbox[2]) // 2
        cy = (bbox[1] + bbox[3]) // 2

        # 1️⃣ 정렬
        if landmarks is not None and len(landmarks) >= 2:
            left_eye, right_eye = landmarks[0], landmarks[1]
            dy, dx = right_eye[1] - left_eye[1], right_eye[0] - left_eye[0]
            angle = np.degrees(np.arctan2(dy, dx))
            M = cv2.getRotationMatrix2D((float(cx), float(cy)), angle, 1.0)
            image = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)

        # 2️⃣ 정사각 크롭
        side = int(max(bbox[2]-bbox[0], bbox[3]-bbox[1]) * 1.3)
        half = side // 2

        x1, y1 = cx - half, cy - half
        x2, y2 = cx + half, cy + half

        px1, py1 = max(0, -x1), max(0, -y1)
        px2, py2 = max(0, x2 - w), max(0, y2 - h)

        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w, x2), min(h, y2)

        crop = image[y1:y2, x1:x2]

        if px1 or py1 or px2 or py2:
            crop = cv2.copyMakeBorder(
                crop, py1, py2, px1, px2,
                cv2.BORDER_CONSTANT, value=[0, 0, 0]
            )

        return cv2.resize(crop, target_size)

    except:
        return None

In [ ]:
def run_preprocessing():
    with zipfile.ZipFile(ZIP_PATH, 'r') as z:
        # 1. 파일 리스트 확인 (디버깅용)
        all_files = z.namelist()
        print(f"📦 압축 파일 내 전체 파일 개수: {len(all_files)}")

        all_videos = [
            f for f in all_files
            if f.lower().endswith('.mp4') and not os.path.basename(f).startswith('._')
        ]
        print(f"🎥 찾은 MP4 영상 개수: {len(all_videos)}")

        if len(all_videos) == 0:
            print("❌ 영상을 찾을 수 없습니다. ZIP_PATH나 파일 확장자를 확인하세요.")
            return

        # 데이터 필터링
        real_videos = [f for f in all_videos if 'celeb-real' in f.lower() or 'youtube-real' in f.lower()]
        fake_all = [f for f in all_videos if 'celeb-synthesis' in f.lower()]

        # 샘플링 개수 확인
        actual_fake_count = min(len(fake_all), FAKE_SAMPLE_COUNT)
        fake_videos = random.sample(fake_all, actual_fake_count)

        print(f"📊 최종 타겟 -> REAL: {len(real_videos)}, FAKE: {len(fake_videos)}")

        for video_list, label_type in [(real_videos, "REAL"), (fake_videos, "FAKE")]:
            if not video_list:
                print(f"⚠️ {label_type} 리스트가 비어있어 건너뜁니다.")
                continue

            print(f"\n🔥 {label_type} 처리 중...")

            for file_path in tqdm(video_list):
                video_name = Path(file_path).stem

                # --- 경로 구조 ---
                if 'youtube-real' in file_path.lower():
                    label_dir = 'real'
                    person_id = 'youtube'
                elif 'celeb-real' in file_path.lower():
                    label_dir = 'real'
                    person_id = video_name.split('_')[0]
                else:
                    label_dir = 'fake'
                    person_id = video_name.split('_')[0]

                save_dir = os.path.join(SAVE_BASE_PATH, label_dir, person_id, video_name)

                # [수정] 덮어쓰기 로직: 존재하면 삭제 후 재생성
                if os.path.exists(save_dir):
                    shutil.rmtree(save_dir)
                os.makedirs(save_dir, exist_ok=True)

                try:
                    with tempfile.NamedTemporaryFile(suffix=".mp4", delete=True) as tmp:
                        tmp.write(z.read(file_path))
                        tmp.flush() # 데이터가 완전히 쓰여졌는지 확인

                        cap = cv2.VideoCapture(tmp.name)
                        total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

                        if total < NUM_FRAMES:
                            cap.release()
                            continue

                        indices = np.linspace(0, total - 1, NUM_FRAMES, dtype=int)
                        frames_cache = []
                        face_crops = []

                        for idx in indices:
                            cap.set(cv2.CAP_PROP_POS_FRAMES, int(idx))
                            ret, frame = cap.read()
                            if not ret or frame is None:
                                continue

                            frames_cache.append(frame)
                            faces = detector.get(frame)

                            if faces:
                                face = max(faces, key=lambda x: (x.bbox[2]-x.bbox[0])*(x.bbox[3]-x.bbox[1]))
                                crop = get_hybrid_face(frame, face)
                                if crop is not None:
                                    face_crops.append(crop)

                        cap.release()

                        # ===== 저장 단계 =====
                        if len(face_crops) == 0:
                            # 전부 실패 → 원본 프레임 리사이즈
                            for i, frame in enumerate(frames_cache[:NUM_FRAMES]):
                                cv2.imwrite(os.path.join(save_dir, f"frame_{i}.jpg"), cv2.resize(frame, TARGET_SIZE))
                        else:
                            # 일부 성공 → 부족한 만큼 마지막 얼굴로 채우기
                            while len(face_crops) < NUM_FRAMES:
                                face_crops.append(face_crops[-1] if face_crops else np.zeros((256,256,3), dtype=np.uint8))

                            for i in range(NUM_FRAMES):
                                cv2.imwrite(os.path.join(save_dir, f"frame_{i}.jpg"), face_crops[i])
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")

                gc.collect()

    print(f"\n✅ 모든 작업이 완료되었습니다. 저장소: {SAVE_BASE_PATH}")
run_preprocessing()

📦 압축 파일 내 전체 파일 개수: 6533
🎥 찾은 MP4 영상 개수: 6529
📊 최종 타겟 -> REAL: 890, FAKE: 890

🔥 REAL 처리 중...


100%|██████████| 890/890 [1:50:42<00:00,  7.46s/it]



🔥 FAKE 처리 중...


100%|██████████| 890/890 [1:50:02<00:00,  7.42s/it]


✅ 모든 작업이 완료되었습니다. 저장소: /content/drive/MyDrive/HECTO/Dataset/Celeb_frames_ver2
